# TO DO
1. we have webscrape the "metacrtics"  website for top 500 movies 

# Task 1 : Webscraping
 1.import required libaries 
    * requests : To retrived the Html of the source page
    * bs4      : To converted the obtained html which in the form of text into soup object in which we can retrive specific                      data like headers, paragraphs, etc.
    * lxml     : Helps bs4 to understand the html.
    * csv      : To write or read into csv file.

In [45]:
import requests
import lxml
import bs4
import csv 
import time


2.Loading the featured page links into a link to hit them

In [56]:
urls =["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4"]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}



3. Initialise the varaible that are used to store the data

In [57]:
movie_short_href={} # used to store the href values 

movie_details={}    # Used to store the final movie details as 
                    #   { 'movie_name':{'Director':['director_name'], 
                    #                   'Principal Cast' : [name_1, name_2 ...name_m], 
                    #                    'Cast':['name_1, name_2... name_n'] }}  note m>n.
            
failed_items=[]     # Used to store the movie if of those movie where we have a failed attempt.




4. Below process is to achieve the link for each movies from the main page

    result : now we have collection of end points of the movies.

In [58]:

index_num=0
for url in urls:
    result = requests.get(url,headers=headers)
    soup = bs4.BeautifulSoup(result.text, "lxml")
    div_tag = soup.find_all('div', class_=["browse_list_wrapper one browse-list-large",
                                           "browse_list_wrapper two browse-list-large",
                                           "browse_list_wrapper three browse-list-large",
                                           "browse_list_wrapper four browse-list-large"])
    for item in div_tag:
        for element in item.find_all('td', class_='clamp-image-wrap'):
            index_num=index_num+1
            source=element.find_all('a' , href= True)[0]
            movie_short_href[index_num]=source['href'].replace('/movie/','')
    
    

In [49]:
url='https://www.metacritic.com/movie/the-gunfighter/details'
result = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
soup = bs4.BeautifulSoup(result.text,'lxml')



5. This is method is to scrape individual movie details from its respective website, if there is any kind of failure while scraping we store the the movie id  in a list (failed_items).

In [59]:
def retrive_data_from_website(i):
    movie_link='https://www.metacritic.com/movie/'
    url = movie_link+movie_short_href[i]+'/details'
    result = requests.get(url,headers=headers)
    if result.status_code != 200:
        failed_items.append(i)
    else:
        
        final_details={}
        soup = bs4.BeautifulSoup(result.text,'lxml')
        tags = soup.find_all('table', class_='credits')
        wanted_columns=['Director', 'Principal Cast', 'Cast']
        movie_name = soup.find_all('div', class_='product_page_title oswald upper')[0].find_all('h1')[0].getText()
        meta_score = soup.find_all('a', class_ = 'metascore_anchor')[0].getText().strip()
        final_details['meta_score']=meta_score
        for table in tags:
            column = table.find_all('th', class_="person")[0].getText()
            if column in wanted_columns:
                details=[]
                for name in table.find_all('a', href=True):
                    details.append(name.getText().strip())
                final_details[column]=details
        if movie_name not in movie_short_href.values():
            movie_short_href[i] = movie_name
        movie_details[movie_short_href[i]] = final_details
    if i in [100,200,300,400,500]:
        time.sleep(2)
    if i in failed_items:
        failed_items.remove(i)
    

6. Now we hit every movie link from our collection and check for any failed attempts, 
    if any there is any failed attempt, we now change their href value in the collection(movie_short_href).

In [60]:

for i in movie_short_href.keys():
    retrive_data_from_website(i)


In [61]:
if len(failed_items)>0:
    for i in failed_items:
        url = 'https://www.metacritic.com/movie/'+movie_short_href[i]
        dummy = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        movie_short_href[i]=dummy.url[dummy.url.rindex('/')+1::]
        retrive_data_from_website(i)

7. Write into a csv file

In [62]:
len(movie_details.keys())

500

In [54]:
for i in  movie_short_href.values() :
    if i in movie_details.keys():
        pass
    else :
        print(i)
        

force-majeure


In [55]:
for k,v in movie_short_href.items():
    if v=='force-majeure':
        print(k)

483


In [63]:
with open("dict.csv", "w", newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in movie_details.items():
        writer.writerow([key, value])



8. Read from a csv file

In [ ]:
with open('dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)


# Load the variable during dev phase 

Need to delete this when the project is completed. copy the cell from variable_pro.ipynb and run it under so we can load movie_details and movie_short_href

# Task
1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?
 to do 
  First extract all cast into a list :
  a) just the Principal cast?
  b) Principal cast +  cast?
  
  Second make a register for cast with their count attached  
  a) list of tuples  [(cast1,count),(cast2, count)...........]
  b) dictionary {{cast1:count}, {cast2: count}......... }
  c) just list  [cast1, cast2....][count1, count 2...] or [cast1, count1, cast2, count2..........]
  

# List the names of the cast that have appeared in different movies
1.Filter all the cast from each movie and append them to the list.

cast_names_all={'name' :{count:'v',
                                rating:'v'
                                }}
                                
                        


In [178]:
cast_details={}
def add_castnames_count(cast_names,score):
    for name in cast_names:
        cast_name_all={}
        if name in cast_details.keys():
            cast_details[name]['count'] = cast_details[name]['count'] + 1
            cast_details[name]['rating'] = cast_details[name]['rating']+int(score)
        else:
            cast_name_all['rating'] = int(score)
            cast_name_all['count'] = 1
            cast_details[name]=cast_name_all
            
        


    


In [179]:
for movie_name,movies in movie_details.items():
    if 'Principal Cast' in movies.keys():
        add_castnames_count(movies['Principal Cast'], movies['meta-score'])
    if 'Cast' in movies.keys():
        add_castnames_count(movies['Cast'],movies['meta-score'])
for name in cast_details.keys():
    cast_details[name]['rating'] = cast_details[name]['rating']/cast_details[name]['count']

In [180]:
cast_details

{'Joseph Cotten': {'rating': 94.2, 'count': 5},
 'Orson Welles': {'rating': 95.0, 'count': 6},
 'Agnes Moorehead': {'rating': 96.5, 'count': 2},
 'Dorothy Comingore': {'rating': 100.0, 'count': 1},
 'Erskine Sanford': {'rating': 96.5, 'count': 2},
 'Everett Sloane': {'rating': 100.0, 'count': 1},
 'Fortunio Bonanova': {'rating': 97.5, 'count': 2},
 'George Coulouris': {'rating': 100.0, 'count': 1},
 'Georgia Backus': {'rating': 94.0, 'count': 3},
 'Gus Schilling': {'rating': 100.0, 'count': 1},
 'Harry Shannon': {'rating': 94.5, 'count': 2},
 'Paul Stewart': {'rating': 93.5, 'count': 2},
 'Philip Van Zandt': {'rating': 100.0, 'count': 1},
 'Ray Collins': {'rating': 96.25, 'count': 4},
 'Ruth Warrick': {'rating': 100.0, 'count': 1},
 'William Alland': {'rating': 100.0, 'count': 1},
 'Al Pacino': {'rating': 91.4, 'count': 5},
 'Marlon Brando': {'rating': 92.57142857142857, 'count': 7},
 'Abe Vigoda': {'rating': 100.0, 'count': 1},
 'Al Lettieri': {'rating': 100.0, 'count': 1},
 'Diane Ke

In [171]:
cast

NameError: name 'cast' is not defined